In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import plotly.express as px
import cv2 as cv

from rover_nerf.feature_map import FeatureMap

# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
global_img = cv.imread('../data/airsim/images/test_scenario.png')
fig = px.imshow(global_img)
fig.update_layout(width=1600, height=800)
fig.show()

In [ ]:
UNREAL_PLAYER_START = np.array([-117252.054688, -264463.03125, 25148.908203])
UNREAL_GOAL = np.array([-83250.0, -258070.0, 24860.0])

start_px = (275, 279)
goal_px = (155, 984)
# start_px = (306, 343)
# goal_px = (207, 1133)

feat_map = FeatureMap(global_img, start_px, goal_px, UNREAL_PLAYER_START, UNREAL_GOAL)

Path comparison

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
xmin, xmax, ymin, ymax = feat_map.bounds
ax.imshow(global_img, extent=feat_map.bounds)
markersize = 1

# Baseline runs
baseline_len = 0
baseline_path_len = 0
baseline_collisions = 0
for i in range(1, 4):
    baseline_data = np.load(f'../data/results/baseline_small_rocks_{i}_states.npz')
    baseline_states = baseline_data['states']
    baseline_len += baseline_states.shape[0]
    baseline_path_len += np.sum(np.linalg.norm(baseline_states[1:-100,:2] - baseline_states[:-101,:2], axis=1))
    baseline_collisions += np.sum(baseline_data['collision_count'])
    ax.plot(baseline_states[:, 0], baseline_states[:, 1], 'b.', markersize=markersize)

# Replan runs
replan_len = 0
replan_path_len = 0
replan_collisions = 0
for i in range(1, 4):
    replan_data = np.load(f'../data/results/run_small_rocks_{i}_states.npz')
    replan_states = replan_data['states']
    replan_len += replan_states.shape[0]
    replan_path_len += np.sum(np.linalg.norm(replan_states[1:-100,:2] - replan_states[:-101,:2], axis=1))
    replan_collisions += np.sum(replan_data['collision_count'])
    ax.plot(replan_states[:, 0], replan_states[:, 1], 'r.', markersize=markersize)

print(f'Baseline: {baseline_len/3} states, {baseline_path_len/3} path length, {baseline_collisions/3} collisions')
print(f'Replan: {replan_len/3} states, {replan_path_len/3} path length, {replan_collisions/3} collisions')


ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
plt.xlabel('x (m)', fontsize=20)
plt.ylabel('y (m)', fontsize=20)
plt.show()

In [ ]:
baseline_data['collision_count']

In [ ]:
replan_data['collision_count']

In [ ]:
# Plot z values
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(baseline_states[:, 2], 'b.', markersize=markersize)
ax.plot(replan_states[:, 2], 'r.', markersize=markersize)
plt.show()

In [ ]:
from rover_nerf.utils import trajectory_plot_trace
import plotly.graph_objects as go

transforms = np.load('transforms.npy')

In [ ]:
N = len(transforms)
Rs = np.zeros((3, 3, N))
ts = np.zeros((N, 3))
for i in range(N):
    Rs[:, :, i] = transforms[i][:3, :3]
    ts[i, :] = transforms[i][:3, 3]

# Plot trajectory
fig = go.Figure(data=trajectory_plot_trace(Rs, ts, scale=0.1))
# fig.add_trace(trajectory_plot_trace(Rs, ts, scale=1.0))
fig.update_layout(width=1500, height=900, scene_aspectmode='data')
fig.show()